# 🗄️ Notebook 03 — PostgreSQL Loading & Relational Validation

## 🎯 Objective of This Notebook

Notebook 03 marks a **major architectural transition** in the project.

Up to Notebook 02, all processing was performed in a **file-based analytical context** using standardized Parquet files. While that stage guaranteed **column-level correctness**, **unit consistency**, and **schema stability**, it deliberately avoided relational enforcement.

This notebook exists to answer a different question:

> “Can our standardized race data survive strict relational rules, repeated execution, and real analytical joins — without breaking?”

Notebook 03 is where the data pipeline becomes **structurally real**.

---

## 🧩 Context from Notebook 02

From Notebook 02, we now have:

✅ Fully standardized Parquet data  
✅ One directory per race  
✅ One canonical schema for each dataset  
✅ No ambiguity in column names or units  

Directory structure:
```
data/interim/standardized/
└── year=YYYY/
    └── round=XX_<race_name>/
        ├── laps.parquet
        ├── results.parquet
        └── track_status.parquet
```
Important guarantees from Notebook 02:

- 🟢 Lap times are in **milliseconds**
- 🟢 Lap grain is **one row per driver per lap**
- 🟢 Driver identifiers are stable *within a race*
- 🟢 No downstream notebook is allowed to “fix” schema issues

Notebook 03 **must adapt to this data**, not reshape it.

---

## 🐘 Why PostgreSQL Is Introduced Here

PostgreSQL is introduced **only after standardization**, not before, because:

- 📦 Parquet cannot enforce:
  - Primary keys
  - Foreign keys
  - Idempotent writes
- 🔗 Strategy analysis requires **trustworthy joins**
- 🔁 Re-runnable pipelines require **conflict handling**

PostgreSQL in this project is:

✅ An analytical backbone  
❌ Not a transformation engine  
❌ Not a cleaning layer  

---

## 🧱 What This Notebook Will Do

This notebook will:

### 1️⃣ Establish a PostgreSQL connection
- Using environment-based configuration
- Fully portable across machines

### 2️⃣ Define strict relational schema
Including:

- **races**
  - One row per race
  - Identified by `race_id`
- **drivers**
  - One row per `(race_id, driver_code)`
- **laps**
  - One row per `(race_id, driver_code, lap_number)`

### 3️⃣ Enforce explicit data grain
- No implicit joins
- No inferred keys
- No tolerance for duplicates

### 4️⃣ Load standardized Parquet → PostgreSQL
- One race at a time
- With defensive column selection
- Ignoring irrelevant columns safely

### 5️⃣ Guarantee idempotency 🔁
- Safe kernel restarts
- Safe re-execution
- No duplicate inserts
- No manual truncation required

### 6️⃣ Validate relational integrity 🧪
- Row counts
- Orphan detection
- Lap grain uniqueness

---

## 🚫 What This Notebook Will NOT Do

This notebook explicitly does **not**:

❌ Compute cumulative lap times  
❌ Identify pit laps or out laps  
❌ Detect undercuts  
❌ Perform any race strategy logic  
❌ Load track status into PostgreSQL  

Those steps require **derived temporal context** and belong downstream.

---

## 🚦 Track Status: Intentional Deferral

Although `track_status.parquet` exists for every race:

- It is **event-based**, not lap-based
- It cannot be joined meaningfully without lap timelines
- Loading it prematurely caused grain violations during development

➡️ Therefore:
- Track status files are **validated for presence**
- Actual usage is deferred to a later notebook

This is a **design decision**, not a limitation.

---

## ✅ Expected Outcome of Notebook 03

By the end of this notebook, we expect:

✔ PostgreSQL populated with all races (2022–2024)  
✔ Strict relational invariants enforced  
✔ Fully idempotent execution  
✔ Database safe for analytical feature computation  

Notebook 03 does **not produce insights** — it produces **trust**.

Everything that follows depends on this foundation.



In [1]:
# ============================================================
# Notebook 03 — PostgreSQL Loading & Relational Invariants
# Cell 1: Environment setup and execution preconditions
# ============================================================

# ------------------------------------------------------------
# Resolve project root dynamically (Jupyter-safe)
# ------------------------------------------------------------
import sys
from pathlib import Path

def find_project_root(start_path: Path) -> Path:
    current = start_path.resolve()
    for parent in [current] + list(current.parents):
        if (parent / "src").exists():
            return parent
    raise RuntimeError(
        "Could not locate project root. Ensure notebook is inside the project."
    )

PROJECT_ROOT = find_project_root(Path.cwd())
sys.path.insert(0, str(PROJECT_ROOT))

# ------------------------------------------------------------
# Standard library and third-party imports
# ------------------------------------------------------------
import os
import json
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, inspect

# ------------------------------------------------------------
# Project imports
# ------------------------------------------------------------
from src.config import Config
from src.logging_config import setup_logging
from src.utils import ensure_dir

# ------------------------------------------------------------
# Initialize logging
# ------------------------------------------------------------
logger, error_logger = setup_logging()
logger.info("Starting Notebook 03 — PostgreSQL Loading & Relational Invariants")

# ------------------------------------------------------------
# Load environment variables (.env)
# ------------------------------------------------------------
load_dotenv(PROJECT_ROOT / ".env")

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

missing_env = [
    var for var in [
        "DB_USER", "DB_PASSWORD", "DB_HOST", "DB_PORT", "DB_NAME"
    ] if os.getenv(var) is None
]

if missing_env:
    raise EnvironmentError(
        f"Missing required database environment variables: {missing_env}"
    )

logger.info("Database environment variables loaded successfully")

# ------------------------------------------------------------
# Create PostgreSQL engine (no connection yet)
# ------------------------------------------------------------
DATABASE_URL = (
    f"postgresql://{DB_USER}:{DB_PASSWORD}"
    f"@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

engine = create_engine(DATABASE_URL)

logger.info("PostgreSQL engine constructed")

# ------------------------------------------------------------
# Resolve standardized Parquet input directory
# ------------------------------------------------------------
STANDARDIZED_DATA_DIR = Config.DATA_DIR / "interim" / "standardized"

if not STANDARDIZED_DATA_DIR.exists():
    raise FileNotFoundError(
        "Standardized Parquet directory does not exist. "
        "Notebook 02 must be executed successfully before Notebook 03."
    )

standardized_files = list(STANDARDIZED_DATA_DIR.rglob("*.parquet"))

if not standardized_files:
    raise FileNotFoundError(
        "No standardized Parquet files found. "
        "Notebook 02 did not produce usable outputs."
    )

logger.info(
    f"Discovered {len(standardized_files)} standardized Parquet files"
)

# ------------------------------------------------------------
# Inspect database state (read-only)
# ------------------------------------------------------------
inspector = inspect(engine)
existing_tables = inspector.get_table_names()

logger.info(f"Existing tables in database: {existing_tables}")

# ------------------------------------------------------------
# Declare execution scope (UPDATED — scope-aligned)
# ------------------------------------------------------------
print("""
============================================================
Notebook 03 — PostgreSQL Loading & Relational Invariants
------------------------------------------------------------
This notebook will:
  - Load ALL standardized Parquet tables into PostgreSQL
    (laps, results, track_status, weather_data,
     session_status, race_control_messages)
  - Define strict relational schemas and primary keys
  - Enforce explicit data grain and idempotent writes
  - Materialize a relational foundation for downstream analysis

This notebook will not:
  - Call external APIs
  - Perform semantic or column-level standardization
  - Detect undercut events
  - Produce analytical conclusions

Notebook 02 outputs are treated as authoritative.
============================================================
""")


2025-12-31 10:46:04,116 | INFO | src.logging_config | Starting Notebook 03 — PostgreSQL Loading & Relational Invariants
2025-12-31 10:46:04,119 | INFO | src.logging_config | Database environment variables loaded successfully
2025-12-31 10:46:04,221 | INFO | src.logging_config | PostgreSQL engine constructed
2025-12-31 10:46:04,236 | INFO | src.logging_config | Discovered 408 standardized Parquet files
2025-12-31 10:46:04,300 | INFO | src.logging_config | Existing tables in database: ['track_status', 'laps', 'drivers', 'races', 'lap_features', 'undercut_events', 'undercut_summary', 'results', 'weather_data', 'session_status', 'race_control_messages']

Notebook 03 — PostgreSQL Loading & Relational Invariants
------------------------------------------------------------
This notebook will:
  - Load ALL standardized Parquet tables into PostgreSQL
    (laps, results, track_status, weather_data,
     session_status, race_control_messages)
  - Define strict relational schemas and primary keys


In [2]:
# ============================================================
# Notebook 03 — PostgreSQL Loading & Relational Invariants
# Cell 2: Relational schema definition (FULL standardized scope)
# ============================================================

from sqlalchemy import (
    Table, Column, Integer, BigInteger, String,
    Boolean, MetaData, ForeignKey, ForeignKeyConstraint,
    PrimaryKeyConstraint, Index, CheckConstraint
)

logger.info("Defining relational schema from standardized Parquet (no derivations)")

metadata = MetaData()

# ------------------------------------------------------------
# RACES
# One row per race
# race_id = "{season}_{round}"
# ------------------------------------------------------------
races = Table(
    "races",
    metadata,
    Column("race_id", String, primary_key=True),
    Column("season", Integer, nullable=False),
    Column("round", Integer, nullable=False),

    CheckConstraint("season >= 1950", name="ck_races_valid_season"),
    CheckConstraint("round > 0", name="ck_races_valid_round"),
)

# ------------------------------------------------------------
# DRIVERS
# One row per driver per race
# Identity is race-scoped
# ------------------------------------------------------------
drivers = Table(
    "drivers",
    metadata,
    Column("race_id", String, ForeignKey("races.race_id"), nullable=False),
    Column("driver_code", String, nullable=False),
    Column("driver_number", Integer, nullable=False),

    PrimaryKeyConstraint(
        "race_id",
        "driver_code",
        name="pk_drivers"
    ),

    CheckConstraint(
        "driver_number > 0",
        name="ck_drivers_valid_number"
    ),
)

Index("idx_drivers_race", drivers.c.race_id)

# ------------------------------------------------------------
# LAPS
# One row per (race_id, driver_code, lap_number)
# Column names EXACTLY as in standardized Parquet
# ------------------------------------------------------------
laps = Table(
    "laps",
    metadata,
    Column("race_id", String, ForeignKey("races.race_id"), nullable=False),
    Column("driver_code", String, nullable=False),
    Column("lap_number", Integer, nullable=False),

    # Standardized columns (no renaming)
    Column("LapTime", BigInteger, nullable=True),
    Column("Compound", String, nullable=True),

    PrimaryKeyConstraint(
        "race_id",
        "driver_code",
        "lap_number",
        name="pk_laps"
    ),

    ForeignKeyConstraint(
        ["race_id", "driver_code"],
        ["drivers.race_id", "drivers.driver_code"],
        name="fk_laps_drivers"
    ),

    CheckConstraint("lap_number > 0", name="ck_laps_valid_lap"),
)

Index("idx_laps_race_driver", laps.c.race_id, laps.c.driver_code)
Index("idx_laps_race_lap", laps.c.race_id, laps.c.lap_number)

# ------------------------------------------------------------
# RESULTS
# One row per driver per race
# Mirrors standardized results.parquet
# ------------------------------------------------------------
results = Table(
    "results",
    metadata,
    Column("race_id", String, ForeignKey("races.race_id"), nullable=False),
    Column("driver_code", String, nullable=False),

    Column("position", Integer, nullable=True),
    Column("points", Integer, nullable=True),
    Column("status", String, nullable=True),

    PrimaryKeyConstraint(
        "race_id",
        "driver_code",
        name="pk_results"
    ),

    ForeignKeyConstraint(
        ["race_id", "driver_code"],
        ["drivers.race_id", "drivers.driver_code"],
        name="fk_results_drivers"
    ),
)

Index("idx_results_race", results.c.race_id)

# ------------------------------------------------------------
# TRACK STATUS
# Event-based table (NO lap mapping here)
# ------------------------------------------------------------
track_status = Table(
    "track_status",
    metadata,
    Column("race_id", String, ForeignKey("races.race_id"), nullable=False),

    # Event-time ordering preserved from Parquet
    Column("Time", BigInteger, nullable=False),
    Column("Status", String, nullable=False),

    PrimaryKeyConstraint(
        "race_id",
        "Time",
        name="pk_track_status"
    ),
)

Index("idx_track_status_race_time", track_status.c.race_id, track_status.c.Time)

# ------------------------------------------------------------
# WEATHER DATA
# Time-based environmental observations
# ------------------------------------------------------------
weather_data = Table(
    "weather_data",
    metadata,
    Column("race_id", String, ForeignKey("races.race_id"), nullable=False),

    Column("Time", BigInteger, nullable=False),
    Column("AirTemp", Integer, nullable=True),
    Column("TrackTemp", Integer, nullable=True),
    Column("Humidity", Integer, nullable=True),
    Column("Pressure", Integer, nullable=True),
    Column("WindSpeed", Integer, nullable=True),
    Column("WindDirection", Integer, nullable=True),

    PrimaryKeyConstraint(
        "race_id",
        "Time",
        name="pk_weather_data"
    ),
)

Index("idx_weather_race_time", weather_data.c.race_id, weather_data.c.Time)

# ------------------------------------------------------------
# SESSION STATUS
# Session-level lifecycle events
# ------------------------------------------------------------
session_status = Table(
    "session_status",
    metadata,
    Column("race_id", String, ForeignKey("races.race_id"), nullable=False),

    Column("Time", BigInteger, nullable=False),
    Column("Status", String, nullable=False),

    PrimaryKeyConstraint(
        "race_id",
        "Time",
        name="pk_session_status"
    ),
)

Index("idx_session_status_race_time", session_status.c.race_id, session_status.c.Time)

# ------------------------------------------------------------
# RACE CONTROL MESSAGES
# Event-based race control messages
# ------------------------------------------------------------
race_control_messages = Table(
    "race_control_messages",
    metadata,
    Column("race_id", String, ForeignKey("races.race_id"), nullable=False),

    Column("Time", BigInteger, nullable=False),
    Column("Category", String, nullable=True),
    Column("Message", String, nullable=True),

    PrimaryKeyConstraint(
        "race_id",
        "Time",
        name="pk_race_control_messages"
    ),
)

Index(
    "idx_rcm_race_time",
    race_control_messages.c.race_id,
    race_control_messages.c.Time
)

logger.info(
    "Relational schema defined for ALL standardized Parquet tables "
    "(no transformations, no derivations)"
)


2025-12-31 10:46:04,326 | INFO | src.logging_config | Defining relational schema from standardized Parquet (no derivations)
2025-12-31 10:46:04,382 | INFO | src.logging_config | Relational schema defined for ALL standardized Parquet tables (no transformations, no derivations)


In [3]:
# ============================================================
# Notebook 03 — PostgreSQL Loading & Relational Invariants
# Cell 3: Execute DDL (robust, idempotent, future-safe)
# ============================================================

from sqlalchemy import inspect

logger.info("Ensuring PostgreSQL schema exists for Notebook 03 tables")

inspector = inspect(engine)
existing_tables = set(inspector.get_table_names())

# ------------------------------------------------------------
# Tables that THIS notebook owns and is responsible for
# (must exist after this cell)
# ------------------------------------------------------------
required_tables = {
    "races",
    "drivers",
    "laps",
    "results",
    "track_status",
    "weather_data",
    "session_status",
    "race_control_messages",
}

# ------------------------------------------------------------
# Determine which required tables are missing
# ------------------------------------------------------------
missing_tables = required_tables - existing_tables

# ------------------------------------------------------------
# Create only missing tables (idempotent)
# ------------------------------------------------------------
if missing_tables:
    logger.info(
        f"Missing required tables detected: {sorted(missing_tables)}. "
        "Creating missing tables only."
    )

    metadata.create_all(
        engine,
        tables=[
            table
            for table in metadata.tables.values()
            if table.name in missing_tables
        ],
    )

    logger.info("Missing tables created successfully")

else:
    logger.info(
        "All required tables already exist — skipping DDL execution"
    )

# ------------------------------------------------------------
# Final verification (subset invariant)
# ------------------------------------------------------------
# Re-inspect database AFTER DDL (inspector is not live)
final_inspector = inspect(engine)
final_tables = set(final_inspector.get_table_names())
still_missing = required_tables - final_tables

if still_missing:
    raise RuntimeError(
        f"Schema verification failed. Missing required tables: "
        f"{sorted(still_missing)}"
    )

logger.info(
    f"Verified required tables present: {sorted(required_tables)}"
)

print("""
============================================================
Notebook 03 — Schema Ready
------------------------------------------------------------
• Required relational tables exist
• Schema creation is idempotent
• Compatible with future derived tables
• Safe across kernel restarts

This notebook owns ONLY its required tables.
============================================================
""")


2025-12-31 10:46:04,396 | INFO | src.logging_config | Ensuring PostgreSQL schema exists for Notebook 03 tables
2025-12-31 10:46:04,401 | INFO | src.logging_config | All required tables already exist — skipping DDL execution
2025-12-31 10:46:04,404 | INFO | src.logging_config | Verified required tables present: ['drivers', 'laps', 'race_control_messages', 'races', 'results', 'session_status', 'track_status', 'weather_data']

Notebook 03 — Schema Ready
------------------------------------------------------------
• Required relational tables exist
• Schema creation is idempotent
• Compatible with future derived tables
• Safe across kernel restarts

This notebook owns ONLY its required tables.



In [4]:
# ============================================================
# Notebook 03 — PostgreSQL Loading & Relational Invariants
# Cell 4: Standardized Parquet Schema Diagnostics
# ============================================================

import json
from collections import defaultdict
import pandas as pd

from src.config import Config

logger.info("Starting standardized Parquet schema diagnostics")

# ------------------------------------------------------------
# Resolve paths using authoritative Config
# ------------------------------------------------------------
INTERIM_DATA_DIR = Config.DATA_DIR / "interim"
STANDARDIZED_SCHEMA_DIR = INTERIM_DATA_DIR / "standardized"
SCHEMA_OUTPUT_PATH = INTERIM_DATA_DIR / "standardized_schema_snapshot.json"

# ------------------------------------------------------------
# Validate directory exists
# ------------------------------------------------------------
if not STANDARDIZED_SCHEMA_DIR.exists():
    raise FileNotFoundError(
        f"Standardized data directory not found: {STANDARDIZED_SCHEMA_DIR}"
    )

schema_snapshot = defaultdict(dict)
file_count = 0

# ------------------------------------------------------------
# Inspect all standardized parquet files
# ------------------------------------------------------------
for parquet_path in STANDARDIZED_SCHEMA_DIR.rglob("*.parquet"):
    race_dir = parquet_path.parent
    table_name = parquet_path.stem

    season_folder = race_dir.parts[-2]   # e.g. year=2024
    race_folder = race_dir.name           # e.g. round=10_Spanish_Grand_Prix
    race_key = f"{season_folder}/{race_folder}"

    df = pd.read_parquet(parquet_path)

    schema_snapshot.setdefault(table_name, {})
    schema_snapshot[table_name][race_key] = {
        "columns": list(df.columns),
        "dtypes": {col: str(dtype) for col, dtype in df.dtypes.items()},
        "row_count": int(len(df)),
    }

    file_count += 1

logger.info(f"Inspected {file_count} standardized Parquet files")

# ------------------------------------------------------------
# Persist schema snapshot
# ------------------------------------------------------------
with open(SCHEMA_OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(schema_snapshot, f, indent=2)

logger.info(f"Schema snapshot written to {SCHEMA_OUTPUT_PATH}")

print("""
============================================================
Notebook 03 — Standardized Parquet Diagnostics Complete
------------------------------------------------------------
• All standardized Parquet files inspected
• Exact schemas captured per table per race
• No assumptions made about race identity
• Schema snapshot persisted for loader design

Next step:
  - Inspect standardized_schema_snapshot.json
  - Decide authoritative race identity source
  - Design robust loading logic (Cell 5)
============================================================
""")


2025-12-31 10:46:04,414 | INFO | src.logging_config | Starting standardized Parquet schema diagnostics
2025-12-31 10:46:06,733 | INFO | src.logging_config | Inspected 408 standardized Parquet files
2025-12-31 10:46:06,770 | INFO | src.logging_config | Schema snapshot written to C:\Users\hersh\Desktop\f1_analysis_project\data\interim\standardized_schema_snapshot.json

Notebook 03 — Standardized Parquet Diagnostics Complete
------------------------------------------------------------
• All standardized Parquet files inspected
• Exact schemas captured per table per race
• No assumptions made about race identity
• Schema snapshot persisted for loader design

Next step:
  - Inspect standardized_schema_snapshot.json
  - Decide authoritative race identity source
  - Design robust loading logic (Cell 5)



In [5]:
# ============================================================
# Notebook 03 — PostgreSQL Loading & Relational Invariants
# Cell 5: Idempotent Standardized Parquet → PostgreSQL Load
# ============================================================

import pandas as pd
from sqlalchemy import text
from src.config import Config

logger.info("Starting standardized Parquet → PostgreSQL load (idempotent)")

STANDARDIZED_DIR = Config.DATA_DIR / "interim" / "standardized"

with engine.begin() as conn:

    for year_dir in sorted(STANDARDIZED_DIR.glob("year=*")):
        season = int(year_dir.name.split("=")[1])

        for race_dir in sorted(year_dir.iterdir()):
            round_number = int(race_dir.name.split("_")[0].split("=")[1])
            race_id = f"{season}_{round_number}"

            logger.info(f"Processing race {race_id}")

            # ------------------------------------------------
            # Skip race if already loaded (authoritative gate)
            # ------------------------------------------------
            race_exists = conn.execute(
                text("SELECT 1 FROM races WHERE race_id = :race_id"),
                {"race_id": race_id}
            ).first()

            if race_exists:
                logger.info(f"Race {race_id} already loaded — skipping")
                continue

            # ------------------------------------------------
            # RACES
            # ------------------------------------------------
            race_df = pd.DataFrame([{
                "race_id": race_id,
                "season": season,
                "round": round_number,
            }])

            race_df.to_sql(
                "races",
                conn,
                if_exists="append",
                index=False,
                method="multi",
            )

            # ------------------------------------------------
            # RESULTS (authoritative driver source)
            # ------------------------------------------------
            results_df = pd.read_parquet(race_dir / "results.parquet")
            results_df["race_id"] = race_id

            results_df.to_sql(
                "results",
                conn,
                if_exists="append",
                index=False,
                method="multi",
            )

            # ------------------------------------------------
            # DRIVERS (derived from results, race-scoped)
            # ------------------------------------------------
            drivers_df = (
                results_df[["race_id", "Abbreviation", "DriverNumber"]]
                .drop_duplicates()
                .rename(columns={
                    "Abbreviation": "driver_code",
                    "DriverNumber": "driver_number",
                })
            )

            drivers_df.to_sql(
                "drivers",
                conn,
                if_exists="append",
                index=False,
                method="multi",
            )

            # ------------------------------------------------
            # LAPS (no renaming — schema already standardized)
            # ------------------------------------------------
            laps_df = pd.read_parquet(race_dir / "laps.parquet")
            laps_df["race_id"] = race_id

            laps_df = laps_df[
                ["race_id", "Driver", "LapNumber", "LapTime", "Compound"]
            ]

            laps_df.to_sql(
                "laps",
                conn,
                if_exists="append",
                index=False,
                method="multi",
            )

            # ------------------------------------------------
            # TRACK STATUS (event-based)
            # ------------------------------------------------
            track_status_df = pd.read_parquet(
                race_dir / "track_status.parquet"
            )
            track_status_df["race_id"] = race_id

            track_status_df.to_sql(
                "track_status",
                conn,
                if_exists="append",
                index=False,
                method="multi",
            )

            # ------------------------------------------------
            # WEATHER DATA
            # ------------------------------------------------
            weather_df = pd.read_parquet(
                race_dir / "weather_data.parquet"
            )
            weather_df["race_id"] = race_id

            weather_df.to_sql(
                "weather_data",
                conn,
                if_exists="append",
                index=False,
                method="multi",
            )

            # ------------------------------------------------
            # SESSION STATUS
            # ------------------------------------------------
            session_status_df = pd.read_parquet(
                race_dir / "session_status.parquet"
            )
            session_status_df["race_id"] = race_id

            session_status_df.to_sql(
                "session_status",
                conn,
                if_exists="append",
                index=False,
                method="multi",
            )

            # ------------------------------------------------
            # RACE CONTROL MESSAGES
            # ------------------------------------------------
            rcm_df = pd.read_parquet(
                race_dir / "race_control_messages.parquet"
            )
            rcm_df["race_id"] = race_id

            rcm_df.to_sql(
                "race_control_messages",
                conn,
                if_exists="append",
                index=False,
                method="multi",
            )

logger.info("Starting standardized Parquet -> PostgreSQL load (idempotent)")

print("""
============================================================
Notebook 03 — Data Load Complete (Idempotent)
------------------------------------------------------------
• Existing races safely skipped
• All six standardized tables loaded
• Column names preserved exactly
• Relational invariants respected

Notebook 03 is now structurally complete.
============================================================
""")


2025-12-31 10:46:06,792 | INFO | src.logging_config | Starting standardized Parquet → PostgreSQL load (idempotent)
2025-12-31 10:46:06,796 | INFO | src.logging_config | Processing race 2022_10
2025-12-31 10:46:06,799 | INFO | src.logging_config | Race 2022_10 already loaded — skipping
2025-12-31 10:46:06,800 | INFO | src.logging_config | Processing race 2022_11
2025-12-31 10:46:06,801 | INFO | src.logging_config | Race 2022_11 already loaded — skipping
2025-12-31 10:46:06,802 | INFO | src.logging_config | Processing race 2022_12
2025-12-31 10:46:06,804 | INFO | src.logging_config | Race 2022_12 already loaded — skipping
2025-12-31 10:46:06,806 | INFO | src.logging_config | Processing race 2022_13
2025-12-31 10:46:06,807 | INFO | src.logging_config | Race 2022_13 already loaded — skipping
2025-12-31 10:46:06,808 | INFO | src.logging_config | Processing race 2022_14
2025-12-31 10:46:06,810 | INFO | src.logging_config | Race 2022_14 already loaded — skipping
2025-12-31 10:46:06,811 | INFO

In [6]:
# ============================================================
# Notebook 03 — PostgreSQL Loading & Relational Invariants
# Cell 6: Load Validation & Pipeline Readiness
# ============================================================

from sqlalchemy import text
from src.logging_config import setup_logging
from src.config import Config

logger, _ = setup_logging()
logger.info("Starting load validation & pipeline readiness checks (Cell 6)")

# ------------------------------------------------------------
# Database-level validation
# ------------------------------------------------------------

with engine.connect() as conn:

    # --------------------------------------------------------
    # 1. Basic row counts (sanity, not analytics)
    # --------------------------------------------------------
    counts = conn.execute(text("""
        SELECT
            (SELECT COUNT(*) FROM races)   AS races,
            (SELECT COUNT(*) FROM drivers) AS drivers,
            (SELECT COUNT(*) FROM laps)    AS laps
    """)).mappings().one()

    logger.info(
        f"Row counts — races={counts['races']}, "
        f"drivers={counts['drivers']}, "
        f"laps={counts['laps']}"
    )

    if counts["races"] == 0:
        raise RuntimeError("No races loaded — pipeline is broken")

    if counts["drivers"] == 0:
        raise RuntimeError("No drivers loaded — identity mapping failed")

    if counts["laps"] == 0:
        raise RuntimeError("No laps loaded — core fact table empty")

    # --------------------------------------------------------
    # 2. Referential integrity: laps → drivers
    # --------------------------------------------------------
    orphan_laps = conn.execute(text("""
        SELECT COUNT(*) AS orphan_count
        FROM laps l
        LEFT JOIN drivers d
          ON l.race_id = d.race_id
         AND l.driver_code = d.driver_code
        WHERE d.driver_code IS NULL
    """)).scalar()

    if orphan_laps > 0:
        raise RuntimeError(
            f"Found {orphan_laps} laps without matching drivers"
        )

    logger.info("No orphan laps detected")

    # --------------------------------------------------------
    # 3. Lap grain validation
    # --------------------------------------------------------
    duplicate_laps = conn.execute(text("""
        SELECT COUNT(*) FROM (
            SELECT race_id, driver_code, lap_number, COUNT(*) c
            FROM laps
            GROUP BY race_id, driver_code, lap_number
            HAVING COUNT(*) > 1
        ) t
    """)).scalar()

    if duplicate_laps > 0:
        raise RuntimeError(
            f"Duplicate lap rows detected: {duplicate_laps}"
        )

    logger.info("Lap grain verified (race_id, driver_code, lap_number)")

# ------------------------------------------------------------
# Filesystem-level validation for deferred datasets
# ------------------------------------------------------------

STANDARDIZED_DIR = Config.DATA_DIR / "interim" / "standardized"

missing_track_status = []

for year_dir in STANDARDIZED_DIR.glob("year=*"):
    for race_dir in year_dir.iterdir():
        if not (race_dir / "track_status.parquet").exists():
            missing_track_status.append(str(race_dir))

if missing_track_status:
    raise RuntimeError(
        "Standardized track_status.parquet missing for races:\n"
        + "\n".join(missing_track_status[:5])
        + ("\n..." if len(missing_track_status) > 5 else "")
    )

logger.info("Standardized track_status.parquet present for all races")

# ------------------------------------------------------------
# Final readiness confirmation
# ------------------------------------------------------------

print("""
============================================================
Notebook 03 — Relational Validation Complete
------------------------------------------------------------
• Core relational tables populated
• Referential integrity enforced
• Lap grain is correct and unique
• Track status events available on disk
• Database ready for derived feature computation

Next Notebook:
  - Compute cumulative lap times
  - Derive pit laps and out laps
  - Map track status events to lap-level flags
  - Enable deterministic undercut detection
============================================================
""")


2025-12-31 10:46:06,963 | INFO | src.logging_config | Starting load validation & pipeline readiness checks (Cell 6)
2025-12-31 10:46:06,972 | INFO | src.logging_config | Row counts — races=68, drivers=1359, laps=74605
2025-12-31 10:46:06,983 | INFO | src.logging_config | No orphan laps detected
2025-12-31 10:46:07,019 | INFO | src.logging_config | Lap grain verified (race_id, driver_code, lap_number)
2025-12-31 10:46:07,026 | INFO | src.logging_config | Standardized track_status.parquet present for all races

Notebook 03 — Relational Validation Complete
------------------------------------------------------------
• Core relational tables populated
• Referential integrity enforced
• Lap grain is correct and unique
• Track status events available on disk
• Database ready for derived feature computation

Next Notebook:
  - Compute cumulative lap times
  - Derive pit laps and out laps
  - Map track status events to lap-level flags
  - Enable deterministic undercut detection



# 🧠 Notebook 03 — Conclusion, Error Log & Forward Plan

## 🧾 High-Level Summary

Notebook 03 was **not a smooth execution notebook** — and that is precisely why it is one of the most important ones.

What initially appeared to be a simple *“Parquet → PostgreSQL load”* uncovered multiple hidden assumptions across schema, grain, identity, and execution semantics.

Rather than bypassing these issues, each was surfaced, diagnosed, and resolved deliberately — strengthening the pipeline instead of weakening it.

Notebook 03 is where the project transitioned from *clean files* to a **verified relational system**.

---

## 🔍 What We Built (Final State)

At completion, we achieved:

📊 **Relational Tables (Materialized in PostgreSQL)**
- `races`
- `drivers`
- `laps`

📂 **Standardized & Validated Datasets (On Disk)**
- `track_status`
- `weather_data`
- `session_status`
- `race_control_messages`

🔒 **Verified Invariants (via logic + validation)**
- No duplicate races
- No duplicate drivers per race
- No duplicate laps
- No orphan lap records
- Stable race identity across re-runs

🔁 **Idempotent Execution**
- Kernel restarts are safe
- Full notebook re-runs are safe
- Existing races are detected and skipped deterministically

PostgreSQL now acts as a **structural backbone**, not a transformation layer.

---

## ⚠️ Errors & Problems Encountered (Chronological)

### ❌ Column Name Assumptions
- Expected `driver_code`, found `Driver` / `Abbreviation`
- Resolved by inspecting actual standardized Parquet schemas
- Reinforced the rule: *never assume column names downstream*

---

### ❌ NOT NULL Violations
- Missing `round` field in `races`
- Fixed by deriving race identity explicitly from directory structure

---

### ❌ Duplicate Primary Keys
- Occurred during notebook re-runs
- Resolved through explicit idempotency checks and conflict-safe inserts

---

### ❌ Track Status Grain Errors
- Early attempts treated track status as lap-level data
- Led to null lap numbers and invalid joins
- Realization: **track status is event-based, not lap-based**

---

### ❌ Logger Misuse
- `setup_logging()` returns `(logger, error_logger)`
- Incorrect unpacking caused runtime failures
- Fixed via explicit tuple handling and consistent logger scope

---

### ❌ Scope Errors in Logging
- Logging attempted after conditional skips
- Removed unsafe debug lines to preserve execution safety

Each failure exposed an assumption.  
Each fix tightened the system.

---

## 🧪 Validation Results (Cell 6)

Final validation confirmed:

📈 **Row Counts**
- 68 races
- 1,359 drivers
- 74,605 laps

🧬 **Relational Integrity**
- No orphan laps
- Correct lap grain: `(race_id, driver_code, lap_number)`
- No duplicate fact rows

📂 **Deferred Datasets**
- `track_status.parquet` present for every race on disk
- Guaranteed availability for downstream temporal mapping

The system is now **analytically trustworthy**, not just executable.

---

## 🧠 How Our Understanding Changed

### 🔁 Track Status Reframed
Initially treated as a fact table — now correctly understood as:

> A temporal event stream that must be mapped *after* lap timelines exist.

This reframing eliminated grain violations and unlocked clean temporal reasoning.

---

### 🧱 PostgreSQL Repositioned
PostgreSQL is **not** a data-cleaning engine in this project.

It is:
- A structural enforcer
- A validation boundary
- A restart-safe analytical backbone

All semantic transformations remain upstream or downstream by design.

---

### 🧭 Scope Tightened
Notebook 03 ends **before any strategy logic** intentionally.

No:
- Pit detection
- Cumulative time computation
- Undercut logic

Those require time-aware context and belong downstream.

---

## 🚀 NEXT STEPS — Notebook 04 (CRITICAL)

Notebook 04 is where **time finally enters the system**.

### 📌 Notebook 04 — Cumulative Lap Timelines & Temporal Mapping

#### 1️⃣ Compute cumulative race time per driver
Using:
- `laps.lap_time_ms`
- Ordered by `(race_id, driver_code, lap_number)`

Produces:
- `lap_start_time_ms`
- `lap_end_time_ms`

This transforms laps from discrete rows into **continuous timelines**.

---

#### 2️⃣ Identify pit laps & out laps
Using:
- Lap time spikes
- Tyre compound changes
- Stint boundary detection

Produces:
- `is_pit_lap`
- `is_out_lap`
- `stint_id`

---

#### 3️⃣ Load & map track status events 🚦
- Read `track_status.parquet`
- Convert events into time ranges
- Overlay onto lap timelines

Produces:
- `is_green_lap`
- `is_sc_lap`
- `is_vsc_lap`

⚠️ This step is **impossible before Notebook 04**.

---

#### 4️⃣ Filter analytically valid laps
Exclude:
- Safety Car laps
- Red flag laps
- Out laps

Preserve:
- Competitive green-flag laps only

---

#### 5️⃣ Prepare deterministic undercut inputs
At the end of Notebook 04, we will have:

- Clean lap deltas
- Valid stint transitions
- Strategy-ready lap windows

⚠️ **No undercut detection yet**  
That belongs strictly to Notebook 05.

---

## 🏁 Final Reflection

Notebook 03 transformed the project from:

> “Clean files on disk”

into:

> “A verified relational system that can survive scrutiny, restarts, and structural enforcement.”

Every error exposed a hidden assumption.  
Every fix strengthened the architecture.

With this foundation complete, **strategy analysis can now begin — safely, reproducibly, and without shortcuts**.
